# CARACTERÍSTICAS DEL CLIENTE - TRATAMIENTO

# Proposito:

En este notebook nos dedicaremos al tratamiento de los diferentes archivos que nos aportan informacion sobre las características de nuestro cliente con el objetivo de establecer un estándar y limpiar los datos para quedarnos con lo que realmente nos interesa. 

In [1]:
%load_ext watermark
%watermark

Last updated: 2023-02-10T12:17:47.316411+00:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 126 Stepping 5, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
import os
import re

# PREPARACION PREVIA

Primero necesitamos copiar todos nuestros archivos en una carpeta donde podramos manipularlos para así conservar los archivos originales

In [3]:
from distutils.dir_util import copy_tree

# Método que limpia la carpeta de datos volatiles ya que es 
# una carpeta activa en todos los programas que hemos generado.
def limpiaCarpetaVolatil(rutaCopia):
    print("Limpiando Carpeta Volatil...")
    for f in os.listdir(rutaCopia):
        os.remove(os.path.join(rutaCopia, f))
        
#Metodo que copia recursivamente
def copia(dir_origen,dir_destino):
    print("Copiando...")
    copy_tree(dir_origen, dir_destino)
    print("Copiado")
    print(os.listdir(dir_destino))
  


In [4]:
rutaBase= "../PredictorGastoTuristico/DatosVirgenes/ISTAC/CaracteristicasCliente/"
rutaCopia= "../PredictorGastoTuristico/DatosVolatiles/"

limpiaCarpetaVolatil(rutaCopia)
copia(rutaBase,rutaCopia)

Limpiando Carpeta Volatil...
Copiando...
Copiado
['turistas_isla.csv', 'turistas_motivos.csv', 'turistas_motivos_isla.csv', 'turistas_origen.csv', 'turistas_paquete.csv', 'turistas_paquete_isla.csv', 'turistas_sexo_edad.csv']


## Primero cargamos los archivos para tratarlos y finalmente unirlos:

In [5]:
ruta = rutaCopia
list_csv_names = os.listdir(ruta)
index = 0
for archivo in list_csv_names:
    print(str(index) + " " + archivo)
    index = index + 1

0 turistas_isla.csv
1 turistas_motivos.csv
2 turistas_motivos_isla.csv
3 turistas_origen.csv
4 turistas_paquete.csv
5 turistas_paquete_isla.csv
6 turistas_sexo_edad.csv


In [6]:
t_isla = pd.read_csv(ruta+list_csv_names[0],keep_default_na=False, na_values=[""])
t_motivos = pd.read_csv(ruta+list_csv_names[1],keep_default_na=False, na_values=[""])
t_motivos_isla = pd.read_csv(ruta+list_csv_names[2],keep_default_na=False, na_values=[""])
t_origen = pd.read_csv(ruta+list_csv_names[3],keep_default_na=False, na_values=[""])
t_paquete = pd.read_csv(ruta+list_csv_names[4],keep_default_na=False, na_values=[""])
t_paquete_isla = pd.read_csv(ruta+list_csv_names[5],keep_default_na=False, na_values=[""])
t_sexo_edad = pd.read_csv(ruta+list_csv_names[6],keep_default_na=False, na_values=[""])

In [7]:
list_all_csv = [t_isla,t_motivos,t_motivos_isla,t_origen,t_paquete,t_paquete_isla,t_sexo_edad]

## Visualizamos el conjunto de todos los csv

Vemos como existen columnas semejantes pero, la fecha por ejemplo no es homogenea

In [8]:
i = 0
for archivo in list_all_csv:
    print("------------------" + list_csv_names[i] +"------------------")
    print(archivo.head(1))
    print("\n")
    i = i +1

------------------turistas_isla.csv------------------
  tipo_turista  periodo   pais       isla   valor
0        Total  2021M11  TOTAL  Lanzarote  205010


------------------turistas_motivos.csv------------------
    pais         motivo  periodo    valor
0  TOTAL  TOTAL MOTIVOS  2021M11  1145484


------------------turistas_motivos_isla.csv------------------
          motivo  periodo   pais       isla   valor
0  TOTAL MOTIVOS  2021M11  TOTAL  Lanzarote  205010


------------------turistas_origen.csv------------------
   periodo   pais    valor
0  2021M11  TOTAL  1145484


------------------turistas_paquete.csv------------------
    pais paquete  periodo    valor
0  TOTAL   Total  2021M11  1145484


------------------turistas_paquete_isla.csv------------------
  paquete  periodo   pais       isla   valor
0   Total  2021M11  TOTAL  Lanzarote  205010


------------------turistas_sexo_edad.csv------------------
    pais   sexo          edad  periodo    valor
0  TOTAL  Total  TOTAL EDADES  

In [9]:
i = 0
for archivo in list_all_csv:
    print("------------------" + list_csv_names[i] +"------------------")
    for columna in archivo:
        print(columna)
        print(archivo[columna].unique())
        print("\n")
    print("\n")
    i = i +1

------------------turistas_isla.csv------------------
tipo_turista
['Total' 'Turistas principales' 'Turistas secundarios']


periodo
['2021M11' '2021M10' '2021M09' '2021M08' '2021M07' '2021M06' '2021M05'
 '2021M04' '2021M03' '2021M02' '2021M01' '2020M12' '2020M11' '2020M10'
 '2020M09' '2020M08' '2020M07' '2020M06' '2020M05' '2020M04' '2020M03'
 '2020M02' '2020M01' '2019M12' '2019M11' '2019M10' '2019M09' '2019M08'
 '2019M07' '2019M06' '2019M05' '2019M04' '2019M03' '2019M02' '2019M01'
 '2018M12' '2018M11' '2018M10' '2018M09' '2018M08' '2018M07' '2018M06'
 '2018M05' '2018M04' '2018M03' '2018M02' '2018M01' '2017M12' '2017M11'
 '2017M10' '2017M09' '2017M08' '2017M07' '2017M06' '2017M05' '2017M04'
 '2017M03' '2017M02' '2017M01' '2016M12' '2016M11' '2016M10' '2016M09'
 '2016M08' '2016M07' '2016M06' '2016M05' '2016M04' '2016M03' '2016M02'
 '2016M01' '2015M12' '2015M11' '2015M10' '2015M09' '2015M08' '2015M07'
 '2015M06' '2015M05' '2015M04' '2015M03' '2015M02' '2015M01' '2014M12'
 '2014M11' '201

['1145484' '1140977' '797652' ... '7586' '4643' '10135']






## 1. FECHA

Vamos a estandarizar la fecha usando los valores trimestrales, pero en el formato estándar de cuartiles. Ya que vemos que tiene el formato "2020 Primer Trimestre"

### 1.1 Transformamos la fecha a un formato usable Year-month

In [10]:
for archivo in list_all_csv:
    archivo.rename(columns={'periodo':'fecha'}, inplace=True)

Vimos como nuestros archivos tienen un formato de fecha de añoMmes, esto el metodo to_datetime de pandas no lo entiende asiqeu debemos cambiarlo por algo que si entienda.

### Aqui qutamos las "M"

Al siguiente metodo se la pasa un dataframe, la columna y el patron que queremos encontrar y remplazar en toda la columna.

In [11]:
# A esta funcion se la pasa un dataframe, la columna y el elemento que queremos encontrar y remplazar en toda la columna
def replace_value(df, df_column_name, patron_to_find, patron_to_replace):
    try:
        my_list = []
        for item in df[df_column_name]: 
            #my_list.append(re.sub(r'(\d+)M(\d+)',r'\1-\2',item))
            my_list.append(re.sub(patron_to_find, patron_to_replace,item))
            
        print("Se ha cambiado el patron ({}) por el patron ({}) en la columna {}".format(
                patron_to_find, patron_to_replace, str(df[df_column_name].name)
            ))
        df[df_column_name] = pd.DataFrame(my_list)
        #print(df.head(5))
    except:
        print("No se ha encontrado el patron")  

Para ello primero cambiamos a tipo string la columna, y tratamos la fecha de todos los archivos

In [12]:
for archivo in list_all_csv:
    archivo.rename(columns={'periodo':'fecha'}, inplace=True)   #Renombramos las columnas
    archivo['fecha'] = archivo['fecha'].astype("string")        #Transformamos la columana a tipo String
    replace_value(archivo, "fecha", r'(\d+)M(\d+)', r'\1-\2')   # llamamos a la funcion que reemplaza nuestro patron

Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha
Se ha cambiado el patron ((\d+)M(\d+)) por el patron (\1-\2) en la columna fecha


Veamos un ejemplo:

In [13]:
list_all_csv[0].head(2)

,tipo_turista,fecha,pais,isla,valor
0,Total,2021-11,TOTAL,Lanzarote,205010
1,Total,2021-11,TOTAL,Fuerteventura,158663


### 1.2 Transformamos a cuartiles

Convertimos de datatime a a quarter

In [14]:
for archivo in list_all_csv:
    archivo.fecha.astype("string")
    archivo['fecha'] = pd.PeriodIndex(archivo.fecha, freq="Q")

Veamos un ejemplo:

In [15]:
list_all_csv[0].head(10)

,tipo_turista,fecha,pais,isla,valor
0,Total,2021Q4,TOTAL,Lanzarote,205010
1,Total,2021Q4,TOTAL,Fuerteventura,158663
2,Total,2021Q4,TOTAL,Gran Canaria,304600
3,Total,2021Q4,TOTAL,Tenerife,476682
4,Total,2021Q4,TOTAL,La Palma,21060
5,Total,2021Q4,TOTAL,Lanzarote,215646
6,Total,2021Q4,TOTAL,Fuerteventura,169817
7,Total,2021Q4,TOTAL,Gran Canaria,292968
8,Total,2021Q4,TOTAL,Tenerife,457826
9,Total,2021Q4,TOTAL,La Palma,15004


### 1.3 Eliminamos cuartiles de 2020 y 2021 al ser anomalos por el covi-19

Vemos el rango que usaremos como filtro

In [16]:
pd.period_range(start=pd.Period('2010Q1', freq='Q'),end=pd.Period('2019Q4', freq='Q'))

PeriodIndex(['2010Q1', '2010Q2', '2010Q3', '2010Q4', '2011Q1', '2011Q2',
             '2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4',
             '2013Q1', '2013Q2', '2013Q3', '2013Q4', '2014Q1', '2014Q2',
             '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3', '2015Q4',
             '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2',
             '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4',
             '2019Q1', '2019Q2', '2019Q3', '2019Q4'],
            dtype='period[Q-DEC]', freq='Q-DEC')

In [17]:
i = 0
for archivo in list_all_csv:
    
    previousShape = archivo.shape
    #Aqui cojo el año y filtro eliminadno 2020 y 2021
    archivo = archivo[archivo.fecha.isin(pd.period_range(start = pd.Period('2010Q1', freq='Q'), 
                                                            end = pd.Period('2019Q4', freq='Q')))]
    archivo.reset_index(drop=True, inplace=True)   #podemos resetear el index del DataFrame
    actualShape = archivo.shape
    print("El archivo tenia (filas,columnas)= {} y ahora tiene (filas,columnas)= {} despues del filtrado".format(
            previousShape,actualShape))
    #Por alguna razon debemos guardar cambios porque de no hacerlo por alguna manera que
    # desconozco no se machaca la informacion existente y no se actualiza el dataframe...
    list_all_csv[i] = archivo
    i = i + 1


El archivo tenia (filas,columnas)= (25740, 5) y ahora tiene (filas,columnas)= (21600, 5) despues del filtrado
El archivo tenia (filas,columnas)= (7150, 4) y ahora tiene (filas,columnas)= (6000, 4) despues del filtrado
El archivo tenia (filas,columnas)= (10725, 5) y ahora tiene (filas,columnas)= (9000, 5) despues del filtrado
El archivo tenia (filas,columnas)= (2288, 3) y ahora tiene (filas,columnas)= (1920, 3) despues del filtrado
El archivo tenia (filas,columnas)= (4290, 4) y ahora tiene (filas,columnas)= (3600, 4) despues del filtrado
El archivo tenia (filas,columnas)= (6435, 5) y ahora tiene (filas,columnas)= (5400, 5) despues del filtrado
El archivo tenia (filas,columnas)= (25740, 5) y ahora tiene (filas,columnas)= (21600, 5) despues del filtrado


In [18]:
## codigo similar pero menos optimo
## i = 0
## for archivo in list_all_csv:
## 
##     #Aqui cojo el año y filtro eliminadno 2020 y 2021
## 
##     archivo["year"] = pd.PeriodIndex(archivo['fecha'], freq="Q").qyear    #cogemos el año y creamos una nueva fila por la que filtrar
##     
##     archivo = archivo[~archivo.year.isin(["2020","2021"])]                #filtramos por fechas que no(~ es la negacion) son 2020 y 2021 
##     
##     archivo.drop('year', axis=1, inplace=True)     #eliminamos la fila year
##     archivo.reset_index(drop=True, inplace=True)   #podemos resetear el index del DataFrame
##     
##     #Por alguna razon debemos guardar cambios porque de no hacerlo por alguna manera que
##     # desconozco no se machaca la informacion existente y no se actualiza el dataframe...
##     list_all_csv[i] = archivo
##     i = i + 1



In [19]:
list_all_csv[0].head()

,tipo_turista,fecha,pais,isla,valor
0,Total,2019Q4,TOTAL,Lanzarote,256733
1,Total,2019Q4,TOTAL,Fuerteventura,168717
2,Total,2019Q4,TOTAL,Gran Canaria,416723
3,Total,2019Q4,TOTAL,Tenerife,526258
4,Total,2019Q4,TOTAL,La Palma,35515


# 2. Estandarización de Islas y Paises 

El estandar establecido es el que esta en el archivo principar que es el gasto_medio_islas.

Este tiene la siguiente notación en las columnas:

Isla: (**YA SE CUMPLE**)
    
    array(['Fuerteventura', 'Gran Canaria', 'La Palma', 'Lanzarote', 'Tenerife', 'CANARIAS'], dtype=object)
       
Pais: (**HAY QUE ESTANDARIZARLO**)

    array(['Alemania', 'España', 'Holanda', 'Otros países', 'Países Nórdicos', 'Reino Unido', 'TOTAL'], dtype=object)

### ESTANDARIZACION PAISES
**Solo estancia_media tiene la columna pais**

vamos a transformar `Dinamarca` `Finlandia` `Suecia` y `Noruega` en `Paises Nordicos` ya que es de donde unico podemos anillarlo con los paises disponibles en el gasto medio

Y `Belgica` `Irlanda` `Suiza` `Italia` `Francia` lo añadiremos a `otros paises`

In [20]:
for archivo in list_all_csv:
    print(archivo.pais.unique())

['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Bélgica' 'Francia'
 'Holanda' 'Irlanda' 'Italia' 'Países Nórdicos' 'Reino Unido'
 'Otros países' 'España']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Irlanda'
 'Países Nórdicos' 'Reino Unido' 'Suiza' 'Otros países' 'España']
['TOTAL' 'España' 'Extranjero']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Bélgica' 'Dinamarca'
 'Finlandia' 'Francia' 'Holanda' 'Irlanda' 'Italia' 'Noruega'
 'Reino Unido' 'Suecia' 'Suiza' 'Otros países' 'España']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Irlanda'
 'Países Nórdicos' 'Reino Unido' 'Suiza' 'Otros países' 'España']
['TOTAL' 'España' 'Extranjero']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Irlanda'
 'Países Nórdicos' 'Reino Unido' 'Suiza' 'Otros países' 'España']


In [21]:
i = 0
for archivo in list_all_csv:
    
    #Primero hay que pasar a str
    archivo.pais.astype(str)                       
    
    #reemplazamos Dinamarca y Suecia por Países Nórdicos
    replace_value(archivo, "pais", r'Dinamarca', r'Países Nórdicos') 
    replace_value(archivo, "pais", r'Suecia', r'Países Nórdicos')
    replace_value(archivo, "pais", r'Finlandia', r'Países Nórdicos')
    replace_value(archivo, "pais", r'Noruega', r'Países Nórdicos')
    
    #reemplazamos 
    replace_value(archivo, "pais", r'Suiza', r'Otros países') 
    replace_value(archivo, "pais", r'Francia', r'Otros países') 
    replace_value(archivo, "pais", r'Bélgica', r'Otros países') 
    replace_value(archivo, "pais", r'Irlanda', r'Otros países') 
    replace_value(archivo, "pais", r'Italia', r'Otros países') 


Se ha cambiado el patron (Dinamarca) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Suecia) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Finlandia) por el patron (Países Nórdicos) en la columna pais


<ipython-input-11-be3c6ce741f8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df_column_name] = pd.DataFrame(my_list)


Se ha cambiado el patron (Noruega) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Suiza) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Francia) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Bélgica) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Irlanda) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Italia) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Dinamarca) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Suecia) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Finlandia) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Noruega) por el patron (Países Nórdicos) en la columna pais
Se ha cambiado el patron (Suiza) por el patron (Otros países) en la columna pais
Se ha cambiado el patron (Francia) por el patron (Otros países) en la colu

In [22]:
#COMPROBAMOS LOS CAMBIOS
for archivo in list_all_csv:
    print(archivo.pais.unique())

['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Otros países' 'Holanda'
 'Países Nórdicos' 'Reino Unido' 'España']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Otros países'
 'Países Nórdicos' 'Reino Unido' 'España']
['TOTAL' 'España' 'Extranjero']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Otros países'
 'Países Nórdicos' 'Holanda' 'Reino Unido' 'España']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Otros países'
 'Países Nórdicos' 'Reino Unido' 'España']
['TOTAL' 'España' 'Extranjero']
['TOTAL' 'RESIDENTES EN EL EXTRANJERO' 'Alemania' 'Holanda' 'Otros países'
 'Países Nórdicos' 'Reino Unido' 'España']


# 3. Astype 

### Columna `valor` tipo int y de paso establecemos las columnas `País, Islas, tipo_turista, motivo, paquete, sexo, edad` como Categóricas


hemos detectado que en la columna valor hay elementos que no son numeros, y son ".", ".." o "...", vamos a sustituir esto por "0"

In [23]:
i = 0
for archivo in list_all_csv:
    
#COLUMNAS COMUNES
    
    #PAIS
    #archivo['pais'] = archivo['pais'].astype("category") -> ERR: SettingWithCopyWarning in Pandas
    archivo = archivo.astype({"pais":'category'}) 
    
    # VALOR
    
    archivo.valor.astype(str)                       #Primero hay que pasar a str ya que hay valores como ".", ".." etc
    replace_value(archivo, "valor", r'\.+', r'0')   #reemplazamos los puntos por 0 porque si no el metodo to_numeric da error
    archivo = archivo.astype({"valor":'int64'}) 
    #archivo['valor'] = pd.to_numeric(archivo.valor.astype(str))  #pasamos la columna a numeros

#COLUMNAS NO COMUNES

    # ISLA
    if 'isla' in archivo.columns:
        archivo = archivo.astype({"isla":'category'})
    # TIPO_TURISTA
    if 'tipo_turista' in archivo.columns:
        archivo = archivo.astype({"tipo_turista":'category'})
    # MOTIVO
    if 'motivo' in archivo.columns:
        archivo = archivo.astype({"motivo":'category'})
    # PAQUETE
    if 'paquete' in archivo.columns:
        archivo = archivo.astype({"paquete":'category'})
    # EDAD
    if 'edad' in archivo.columns:
        archivo = archivo.astype({"edad":'category'})
    # SEXO
    if 'sexo' in archivo.columns:
        archivo = archivo.astype({"sexo":'category'})
    
    print("Se han modificado los dtypes del archivo: {} a: \n {}".format(
            list_csv_names[i], archivo.dtypes
    ))
     
    #Por alguna razon debemos guardar cambios porque de no hacerlo por alguna manera que
    # desconozco no se machaca la informacion existente y no se actualiza el dataframe...
    list_all_csv[i] = archivo
    i = i + 1

Se ha cambiado el patron (\.+) por el patron (0) en la columna valor
Se han modificado los dtypes del archivo: turistas_isla.csv a: 
 tipo_turista         category
fecha           period[Q-DEC]
pais                 category
isla                 category
valor                   int64
dtype: object
Se ha cambiado el patron (\.+) por el patron (0) en la columna valor
Se han modificado los dtypes del archivo: turistas_motivos.csv a: 
 pais           category
motivo         category
fecha     period[Q-DEC]
valor             int64
dtype: object
Se ha cambiado el patron (\.+) por el patron (0) en la columna valor
Se han modificado los dtypes del archivo: turistas_motivos_isla.csv a: 
 motivo         category
fecha     period[Q-DEC]
pais           category
isla           category
valor             int64
dtype: object
Se ha cambiado el patron (\.+) por el patron (0) en la columna valor
Se han modificado los dtypes del archivo: turistas_origen.csv a: 
 fecha    period[Q-DEC]
pais          catego

In [24]:
i = 0
for archivo in list_all_csv:
    print(archivo.dtypes)

tipo_turista         category
fecha           period[Q-DEC]
pais                 category
isla                 category
valor                   int64
dtype: object
pais           category
motivo         category
fecha     period[Q-DEC]
valor             int64
dtype: object
motivo         category
fecha     period[Q-DEC]
pais           category
isla           category
valor             int64
dtype: object
fecha    period[Q-DEC]
pais          category
valor            int64
dtype: object
pais            category
paquete         category
fecha      period[Q-DEC]
valor              int64
dtype: object
paquete         category
fecha      period[Q-DEC]
pais            category
isla            category
valor              int64
dtype: object
pais          category
sexo          category
edad          category
fecha    period[Q-DEC]
valor            int64
dtype: object


## 4. Unión de cuartiles

### 4.1 Prueba

Haremos una prueba con lo que queremos unir para ello cogeremos el primer archivo y miraremos los datos que queremos unir como ejemplo usaremos Alemania en 2019 el total de turistas por Gran Canaria

In [25]:
df = list_all_csv[0]
print(df.head(10))
print(df.shape)
print(df.dtypes)

  tipo_turista   fecha   pais           isla   valor
0        Total  2019Q4  TOTAL      Lanzarote  256733
1        Total  2019Q4  TOTAL  Fuerteventura  168717
2        Total  2019Q4  TOTAL   Gran Canaria  416723
3        Total  2019Q4  TOTAL       Tenerife  526258
4        Total  2019Q4  TOTAL       La Palma   35515
5        Total  2019Q4  TOTAL      Lanzarote  231995
6        Total  2019Q4  TOTAL  Fuerteventura  159352
7        Total  2019Q4  TOTAL   Gran Canaria  405715
8        Total  2019Q4  TOTAL       Tenerife  487576
9        Total  2019Q4  TOTAL       La Palma   29614
(21600, 5)
tipo_turista         category
fecha           period[Q-DEC]
pais                 category
isla                 category
valor                   int64
dtype: object


Filtramos:

In [26]:
#df.loc[df['Grade'].isin(['A', 'B']),['Name','TotalMarks', 'Grade'] ]
x = df[df.fecha.isin(pd.period_range('2019Q4', periods=1, freq='Q')) 
           & df.pais.isin(["Alemania"])
           & df.isla.isin(["Gran Canaria"])
           & df.tipo_turista.isin(["Total"])
           ]
#x = df[df.fecha.isin(pd.period_range('2019Q4', periods=1, freq='Q'))]
#x = df[df.pais.isin(["Alemania"])]
#x = df[df.isla.isin(["Gran Canaria"])]
#pd.PeriodIndex("2019Q4", freq="Q")
x.head(10)

,tipo_turista,fecha,pais,isla,valor
1202,Total,2019Q4,Alemania,Gran Canaria,101422
1207,Total,2019Q4,Alemania,Gran Canaria,94554
1212,Total,2019Q4,Alemania,Gran Canaria,62994


Aquí vemos los valores que si lo sumamos tenemos 258970 y veremos mas adelante como este numeor coincide, por lo que la union se hace correctamente

Para unirlo haremos un groupby() por los nombres de todas las columnas que queremos considerar a la hora de ver dos filas como iguales. Las consideraremos todas, excepto valor. Una vez agrupadas, te quedas con la columna valor para aplicar sobre ella la función sum.

El resultado sería una serie con las sumas de "valor", pero indexado jerárquicamente (multi-index) según los grupos formados. Hacemos un .reset_index() para "deshacer" el índice jerárquico, convirtiéndolo de nuevo en columnas como el que tenía el dataframe original.

In [27]:
res = df.groupby(["tipo_turista","fecha","pais","isla"]).valor.apply(sum).reset_index()

In [28]:
print(res.head(10))
print(res.shape)

  tipo_turista   fecha      pais           isla   valor
0        Total  2010Q1  Alemania  Fuerteventura  143560
1        Total  2010Q1  Alemania   Gran Canaria  213576
2        Total  2010Q1  Alemania       La Palma   24677
3        Total  2010Q1  Alemania      Lanzarote   78239
4        Total  2010Q1  Alemania       Tenerife  166119
5        Total  2010Q1    España  Fuerteventura   26503
6        Total  2010Q1    España   Gran Canaria   91298
7        Total  2010Q1    España       La Palma   11051
8        Total  2010Q1    España      Lanzarote   70319
9        Total  2010Q1    España       Tenerife  150615
(4800, 5)


In [29]:
res.dtypes

tipo_turista           object
fecha           period[Q-DEC]
pais                   object
isla                   object
valor                   int64
dtype: object

In [30]:
x = res[res.fecha.isin(pd.period_range('2019Q4', periods=1, freq='Q')) 
           & res.pais.isin(["Alemania"])
           & res.isla.isin(["Gran Canaria"])
           & res.tipo_turista.isin(["Total"])
           ]
x.head()

,tipo_turista,fecha,pais,isla,valor
1561,Total,2019Q4,Alemania,Gran Canaria,258970


### 4.2 Aquí creamos el metodo para unir los datos en todos los ficheros

In [31]:
# A este método se le pasa como parámetro un dataframe y agrupa las filas cuyos valores por columnas 
# son iguales menos la columna "valor" para el que se sumaran los valores
def juntar(df):
    #Creamos un array con todas las columnas menos valor que es comun a todas y es por donde uniremos
    lastshape = df.shape
    arr =[]
    for colum in df:
        if colum != "valor": 
            arr.append(colum)
    
    
    df = df.groupby(arr).valor.apply(sum).reset_index()
    newshape = df.shape
    
    print("El archivo antes tenia las filas y columnas {} ahora depues de unir por los cuartiles tiene {}".format(
            lastshape, newshape
    ))
    print(df.head(2))
    return df

# EJECUTAMOS LA FUNCIÓN JUNTAR() Y GUARDAMOS LOS CAMBIOS

In [32]:
i=0
for archivo in list_all_csv:
    archivo = juntar(archivo)
    #Guardamos
    rutaDatosTratados = "../PredictorGastoTuristico/DatosGuardados/"
    archivo.to_csv(rutaDatosTratados + list_csv_names[i] + ".csv", index = False, encoding='utf-8')
    i = i + 1

El archivo antes tenia las filas y columnas (21600, 5) ahora depues de unir por los cuartiles tiene (4800, 5)
  tipo_turista   fecha      pais           isla   valor
0        Total  2010Q1  Alemania  Fuerteventura  143560
1        Total  2010Q1  Alemania   Gran Canaria  213576
El archivo antes tenia las filas y columnas (6000, 4) ahora depues de unir por los cuartiles tiene (1600, 4)
       pais             motivo   fecha   valor
0  Alemania  Ocio o vacaciones  2010Q1  604627
1  Alemania  Ocio o vacaciones  2010Q2  460804
El archivo antes tenia las filas y columnas (9000, 5) ahora depues de unir por los cuartiles tiene (3000, 5)
              motivo   fecha    pais           isla  valor
0  Ocio o vacaciones  2010Q1  España  Fuerteventura  22685
1  Ocio o vacaciones  2010Q1  España   Gran Canaria  57676
El archivo antes tenia las filas y columnas (1920, 3) ahora depues de unir por los cuartiles tiene (320, 3)
    fecha      pais   valor
0  2010Q1  Alemania  620920
1  2010Q1    España  3